# Monte Carlo Methods 1

## Contents
* [Acknowledgements](#ackw)
* [Overview](#overview) 
* [Monte Carlo methods](#ekf)
    * [Example: Tossing a coin 4 times](#test_case_1)
    * [Areas of applicability](#test_case_2)
        * [Estimate probabilities](#test_case_2_1)
    * [Size of $N$](#test_case_3)
        * [Example: Shared computer](#test_case_3_1)
    * [Example: New software release](#test_case_4)
* [References](#refs)

## <a name="ackw"></a> Acknowledgements

## <a name="overview"></a> Overview

This notebook introduces the <a href="https://en.wikipedia.org/wiki/Monte_Carlo_method"> Monte Carlo</a> method or MC for short. Monte Carlo methods are mostly used for the computation of probabilities, expected values,
and other distribution characteristics [1]. 


## <a name="overview"></a> Monte Carlo methods

The  general idea behind MC methods is the sampling of a process in order to determine some statistical properties. 
Recall that probability can be defined as a long-run proportion. With the help of random number generators, computers can actually simulate a long run. Then, the probability can be estimated via the  observed frequency [1]. The longer the run is simulated, the more accurate result is obtained.

Consider for example the case where we toss a coin four times. We want to calculate the probability that the outcome will be 3 heads and 1 tail. We know how to calculate this probability using the formula below

$$P(3H) = \begin{pmatrix}4 \\ 3\end{pmatrix} \left(\frac{1}{2}\right)^3\left(1 - \frac{1}{2}\right)^1 = \frac{1}{4}$$

However, we can simulation to calculate this as the next example shows.

### <a name="test_case_1"></a> Example: Tossing a coin 4 times

In [1]:
from random import randint

In [6]:
def simulate(nsims):
    success = 0
    
    for i in range(nsims):
        if randint(0,1) + randint(0,1) + randint(0,1) + randint(0,1) == 3:
            success += 1
            
    print("Number of simulations: ",nsims)
    print("Number of success: ", success)
    
    

In [7]:
simulate(10)
simulate(100)
simulate(1000)
simulate(10000)
simulate(100000)

Number of simulations:  10
Number of success:  2
Number of simulations:  100
Number of success:  21
Number of simulations:  1000
Number of success:  262
Number of simulations:  10000
Number of success:  2414
Number of simulations:  100000
Number of success:  25097


As the number of iterations increases the algorithm converges towards the correct answer of 0.25.

However, we need more than that in order to convence ourselves for the effort. Let's see some areas of application of the MCM.

### <a name="test_case_2"></a> Areas of applicability

MC moethos have various fields of application. Just to mention a few, see also [1], forecasting, percolation, queuing and Markov chain MC.

Below, we discuss one of the most basic MC applications namely probability estimation.

#### <a name="test_case_2_1"></a> Estimate probabilities

The most basic application of MC methods is in the estiamtion of probabilities. Recall, that probabilities can be viewed as long-run proportions. Thus, we can generate a long-run Of experiments and compute the proportion of times when our event occured. 

Assume that the random variable $X$  follows the probability distribution $F$. Then we can compute the probability $P(X \in A)$, for some set $A$, via the formula

$$P(X \in A) = \frac{\text{number of} ~ X_i \in A}{N}$$

where $N$ is the size of  a  MC  experiment,  $X_i$ are  generated  random variables with the same distribution as $X$. Note that this is just an estimation and following common statistic literature we should  instead write


$$\hat{P}(X \in A) = \frac{\text{number of} ~ X_i \in A}{N}$$

Whenever we deal with an estimation method, we should ask how much accurate this is. In order to answer this question we must compute $E[\hat{P}]$ and the $Var[\hat{P}]$ [1]. The number of $X_i$ that fall within the set $A$ follows the <a href="https://en.wikipedia.org/wiki/Binomial_distribution">Binomial distribution</a> with parameters $N,p$.  Thus, 

$$E[\hat{P}] = \frac{1}{N}Np = p$$

$$Var[\hat{P}] = \frac{1}{N^2}Npq = \frac{1}{N}pq$$

From the variance formula, we see that the variance, i.e. the error, of the estimator decreases with $N$. We also see that our estimator is an <a href="https://en.wikipedia.org/wiki/Bias_of_an_estimator">unbiased estimator</a>.

### <a name="test_case_3"></a> Size of $N$

The question that pops up naturally is how large $N$ should be for a given error margin $\epsilon$. Let's how we can do this. To start with, the variable, see [1], the following variable follows the standard normal distribution:

$$\frac{\hat{p}-p}{\sqrt{\frac{p(1-p)}{N}}} \sim N(0,1)$$

therefore, 

$$P(|\hat{p}-p| > \epsilon) \approx 2 \Phi\left(- \frac{\epsilon \sqrt{N}}{\sqrt{p(1-p)}}\right)$$

This is simply the probability of the error exceeding $\epsilon$. We can design a MC simulation that attains this accuracy by choosing $N$ so that the error does not exceed $\epsilon$ with probability $(1-\alpha)$ i.e. we will find $N$ so that

$$P(|\hat{p}-p| > \epsilon) \leq \alpha$$

Assume that we use an intelligent guess for $p$ say $\bar{p}$. Then

$$2 \Phi\left(- \frac{\epsilon \sqrt{N}}{\sqrt{\bar{p}(1-\bar{p})}}\right) \leq \alpha$$

solving this inequality leads to:

$$N \geq \bar{p}(1-\bar{p})\left(\frac{z_{\alpha/2}}{\epsilon}\right)^2$$

Alternatively, we could also bound the term $p(1-p)\leq 0.25, 0 \leq p \leq 1$ and solve

$$2 \Phi\left(- 2\epsilon \sqrt{N}\right) \leq \alpha$$

which leads to 

$$N \geq 0.25\left(\frac{z_{\alpha/2}}{\epsilon}\right)^2$$

#### <a name="test_case_3_1"></a> Example: Shared computer

This example is edited from example 5.14 from [1]. A supercomputer is shared by 250 independent subscribers. Each day, each subscriber uses
the facility with probability 0.3. The number of tasks sent by each active user follows a <a href="https://en.wikipedia.org/wiki/Uniform_distribution_(continuous)">uniform distribution</a> $U(1, 50)$. The time each task takes, follows a uniform distribution with $U(0.01, 1.5)$ in minutes. Tasks are processed consecutively. What is the probability that all the
tasks will be processed, that is, the total requested computer time is less than 24 hours?
Estimate this probability, attaining the margin of error $\pm 0.01$ with probability 0.99.


Assuming that we have in hand an $N$ that satisfies the marginal error, $\pm 0.01$,  with the given probability, 0.99, we can proceed as follows. For every iteration generate the number of users that are active. This number follows a <a href="https://en.wikipedia.org/wiki/Binomial_distribution">Binomial distribution</a>, $B(n,p)$ where $n=250$ and $p=0.3$. For each active user we generate the number of tasks he is going to submit. This follows a uniform distribution. For each task we compute the time it takes to finish and sum this into a global variable. 

We can compute $N$ using 

$$N \geq 0.25\left(\frac{z_{\alpha/2}}{\epsilon}\right)^2$$

as we don't have an intelligent guess. Thus $N=0.25\left(\frac{2.575}{0.01}\right)^2 = 16577$

In [15]:
import numpy as np
import random
import math

N=16577
total_time = [0.0]*N
C = 250
p = 0.3
a = 0.01
b = 1.5 

for itr in range(N):
    
    itr_time = 0.0
    
    # generate the number of active users
    n_users = 0
    for u in range(C):
        
        experiment = np.random.binomial(size=1, n=1, p=p)
        
        if experiment == 1:
            n_users += 1
    
    #print("For iteration {0} number of users {1}".format(itr, n_users))
    for user in range(n_users):
        
        # for each active user generate the total 
        # number of tasks he submits.
        n_total_tasks = random.randint(1, 50)
        
        for task in range(n_total_tasks):
            itr_time += np.random.uniform(a, b)
    
    total_time[itr] = itr_time



In [16]:
# the established probability is the proportion
# of T<1440 over all N
counter = 0
for t in range(len(total_time)):
    if total_time[t]< 1440.0:
        counter += 1
        
        
print("p_hat is: ", counter/len(total_time))

p_hat is:  0.49960789045062437


### <a name="test_case_4"></a> New software release

Let's see one more example of how MC methods can be used. This is example 5.16 page 119 from [1]. 

The example develops a simple stochastic model for the number of errors in a new software. Every day the developers find a random number of errors and fix them. Let $X_t$ denote the number of errors found on day $t$. Assume that $X_t \sim Poisson(\lambda_t)$ where the Poissong parameter is chosen as the lowest number of errors found during the previous three days i.e.

$$\lambda_t = min (X_{t-1}, X_{t-2}, X_{t-3})$$

Assume that the first three days the developers found 28, 22 and 18 errors. We want to:

- Predict the time it will take to find all the errors.
- Estimate the probability that some errors will remain undetected after 21 days.
- Predict the total number of errors in the release.


Below is the Python script that executes the MC simulation. We use $N=1000$ runs. 

In [32]:
import numpy as np
from scipy.stats import poisson
import random

N=1000
time=[0.0]*N
errors=[0]*N

for itr in range(N):
    
    last=[28, 22, 18]
    detected_errors = sum(last)
    
    t=0
    x = min(last)
    
    while x > 0:
        
        l = min(last)
        x = poisson.rvs(l, size=1)
        
        detected_errors += x
        t += 1
        
        # update how we choose lambda
        last = last[1:3] 
        last.append(x)
    time[itr] = t - 1
    errors[itr] = detected_errors

In [33]:
#  the expected time it takes to detect all the errors 
print("Expected detection time: ", np.mean(time))

# expected total number of errors
print("Expected total number of errors: ", np.mean(errors))

Expected detection time:  16.5
Expected total number of errors:  215.938


## <a name="refs"></a> References

1. Michael Baron, ```Probability  and Statistics for Computer Scientists.``` CRC Press. 
2. Reuven Y. Rubinstein, Dirk P. Kroese ```SIMULATION AND THE MONTE CARLO METHOD, Third Edition.``` Wiley.


